In [9]:
from utils import *
def MessageClassifier_with_1gram_for_Prediction(username, prname):
    if (username == None) & (prname == None):
        input_directory=''
    else:    
        input_directory = "/".join([username, prname])
    
    #ref: 학습한 모델 이름 (.h5는 제외하고),vocab_size,maxlen
    #ref 주의할점!!!! 각 세개를 띄워서 쓰면 안됨!! 오류남
    #input_: 예측할 별점 없는 데이터 파일 이름 
    #output_ : 예측 점수가 들어가있는 결과 데이터프레임 파일 이름 (/xlsx)
    ref, input_, output_ = Read_Arg_(username, prname, "MessageClassifier_with_1gram_for_Prediction")
    
    option_dic=option_finder(username, prname, "MessageClassifier_with_1gram_for_Prediction")
    print(ref)
    print(input_)
    print(option)        
if __name__=='__main__':
    MessageClassifier_with_1gram_for_Prediction(None,None)


네이버맵리뷰통합.xlsx
{'option_1': 80, 'option_2': 'without3pt.h5', 'option_3': 'tokenizer.pickle'}


#### 교수님 의견 반영한 코드  

In [17]:
%%writefile MessageClassifier_with_1gram_for_Prediction.py
#MessageClassifier_with_1gram_for_Prediction.py
from utils import *
import re
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()
def MessageClassifier_with_1gram_for_Prediction(username, prname):
    
    # Create new `pandas` methods which use `tqdm` progress
    # (can use tqdm_gui, optional kwargs, etc.)
    tqdm.pandas()
    #0. 파일 읽기 
    if (username == None) & (prname == None):
        input_directory=''
    else:    
        input_directory = "/".join([username, prname])
    
    #ref: 학습한 모델 이름 (.h5는 제외하고),vocab_size,maxlen
    #ref 주의할점!!!! 각 세개를 띄워서 쓰면 안됨!! 오류남
    #input_: 예측할 별점 없는 데이터 파일 이름 
    #output_ : 예측 점수가 들어가있는 결과 데이터프레임 파일 이름 (/xlsx)
    ref, input_, output_ = Read_Arg_(username, prname, "MessageClassifier_with_1gram_for_Prediction")
    #1. 메캅설치 확인 (형태소 분석기 어느형태든 상관없음)
    try: 
        mecab = Mecab("C:\mecab\mecab-ko-dic") #자신의 메캅설치 경로 
        print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))
    except :
        print('메캅이 설치되어 있지 않습니다')
    
    #2. 데이터 읽기
    if input_[-3:]=='csv':
        df = pd.read_csv(input_)
    elif input_[-4:]=='xlsx':
        df = pd.read_excel(input_)
    else:
        print('잘못된 인풋파일 형식입니다.')
    
    #'No contents' 행 제거해버리기 
    df=df.dropna()
    index_No= df[df['contents']=='No Contents'].index
    df= df.drop(index_No)
    
    #3. 학습했던 모델, 토크나이저  불러오기
    option_dic=option_finder(username, prname, "MessageClassifier_with_1gram_for_Prediction")
    loaded_model = load_model(option_dic['option_1'])
    with open(option_dic['option_2'], 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    #4. cutoff 가 0.6인 긍부정 예측 함수 
    def sentiment_predict2(new_sentence):
        stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
        new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
        new_sentence = mecab.morphs(new_sentence)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
        if option_dic['option_3']=='default length':
            max_len= 80
        else:
            max_len= int(option_dic['option_3'])
        pad_new = pad_sequences(encoded, max_len )

        score = float(loaded_model.predict(pad_new))
        if(score > 0.6):
            #print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
            return (1)
        else:
            #print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
            #print(score)
            return (0)

    pred_df=pd.DataFrame(columns=['predict_value'])
    df['contents']= df['contents'].astype(str)
    pred_df['predict_value'] = df['contents'].progress_apply(sentiment_predict2)
    output_df = pd.concat([df['user'],pred_df['predict_value']],axis=1)
    #output_df = pd.concat([df['contents'],pred_df['predict_value']],axis=1)
    output_df.to_excel(output_)
if __name__=='__main__':
    MessageClassifier_with_1gram_for_Prediction(None,None)

Overwriting MessageClassifier_with_1gram_for_Prediction.py


#### 원래 GRU 코드  

In [9]:
#GRU_pos_neg_predict.py
from utils import *
import re
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()
def GRU_pos_neg_predict(username, prname):
    
    # Create new `pandas` methods which use `tqdm` progress
    # (can use tqdm_gui, optional kwargs, etc.)
    tqdm.pandas()
    #0. 파일 읽기 
    if (username == None) & (prname == None):
        input_directory=''
    else:    
        input_directory = "/".join([username, prname])
    
    #ref: 학습한 모델 이름 (.h5는 제외하고),vocab_size,maxlen
    #ref 주의할점!!!! 각 세개를 띄워서 쓰면 안됨!! 오류남
    #input_: 예측할 별점 없는 데이터 파일 이름 
    #output_ : 예측 점수가 들어가있는 결과 데이터프레임 파일 이름 (/xlsx)
    ref, input_, output_ = Read_Arg_(username, prname, "GRU_pos_neg_predict")
    #1. 메캅설치 확인 (형태소 분석기 어느형태든 상관없음)
    try: 
        mecab = Mecab("C:\mecab\mecab-ko-dic") #자신의 메캅설치 경로 
        print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))
    except :
        print('메캅이 설치되어 있지 않습니다')
    
    #2. 데이터 읽기
    if input_[-3:]=='csv':
        df = pd.read_csv(input_)
    elif input_[-4:]=='xlsx':
        df = pd.read_excel(input_)
    else:
        print('잘못된 인풋파일 형식입니다.')
    df=df.iloc[98333:400000]
    #3. 학습했던 모델, 토크나이저  불러오기
    ref_list= ref.split(',')
    loaded_model = load_model(ref_list[0])
    with open(ref_list[1], 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    #4. cutoff 가 0.6인 긍부정 예측 함수 
    def sentiment_predict2(new_sentence):
        stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
        new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
        new_sentence = mecab.morphs(new_sentence)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
        pad_new = pad_sequences(encoded, maxlen = int(ref_list[2]))

        score = float(loaded_model.predict(pad_new))
        if(score > 0.6):
            #print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
            return (1)
        else:
            #print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
            #print(score)
            return (0)

    pred_df=pd.DataFrame(columns=['predict_value'])
    df['contents']= df['contents'].astype(str)
    pred_df['predict_value'] = df['contents'].progress_apply(sentiment_predict2)
    output_df = pd.concat([df['user'],pred_df['predict_value']],axis=1)
    #output_df = pd.concat([df['contents'],pred_df['predict_value']],axis=1)
    output_df.to_excel(output_)
if __name__=='__main__':
    GRU_pos_neg_predict(None,None)

['와', '이런', '것', '도', '상품', '이', '라고', '차라리', '내', '가', '만드', '는', '게', '나을', '뻔']


100%|████████████████████████████████████████████████████████████████████████| 301667/301667 [2:24:31<00:00, 34.79it/s]


In [1]:
#GRU_pos_neg_predict.py
from utils import *
import re
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()
def GRU_pos_neg_predict(username, prname):
    
    # Create new `pandas` methods which use `tqdm` progress
    # (can use tqdm_gui, optional kwargs, etc.)
    tqdm.pandas()
    #0. 파일 읽기 
    if (username == None) & (prname == None):
        input_directory=''
    else:    
        input_directory = "/".join([username, prname])
    
    #ref: 학습한 모델 이름 (.h5는 제외하고),vocab_size,maxlen
    #ref 주의할점!!!! 각 세개를 띄워서 쓰면 안됨!! 오류남
    #input_: 예측할 별점 없는 데이터 파일 이름 
    #output_ : 예측 점수가 들어가있는 결과 데이터프레임 파일 이름 (/xlsx)
    ref, input_, output_ = Read_Arg_(username, prname, "GRU_pos_neg_predict")
    #1. 메캅설치 확인 (형태소 분석기 어느형태든 상관없음)
    try: 
        mecab = Mecab("C:\mecab\mecab-ko-dic") #자신의 메캅설치 경로 
        print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))
    except :
        print('메캅이 설치되어 있지 않습니다')
    
    #2. 데이터 읽기
    if input_[-3:]=='csv':
        df = pd.read_csv(input_)
    elif input_[-4:]=='xlsx':
        df = pd.read_excel(input_)
    else:
        print('잘못된 인풋파일 형식입니다.')
    df=df.iloc[400000:]
    #3. 학습했던 모델, 토크나이저  불러오기
    ref_list= ref.split(',')
    loaded_model = load_model(ref_list[0])
    with open(ref_list[1], 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    #4. cutoff 가 0.6인 긍부정 예측 함수 
    def sentiment_predict2(new_sentence):
        stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
        new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
        new_sentence = mecab.morphs(new_sentence)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
        pad_new = pad_sequences(encoded, maxlen = int(ref_list[2]))

        score = float(loaded_model.predict(pad_new))
        if(score > 0.6):
            #print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
            return (1)
        else:
            #print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
            #print(score)
            return (0)

    pred_df=pd.DataFrame(columns=['predict_value'])
    df['contents']= df['contents'].astype(str)
    pred_df['predict_value'] = df['contents'].progress_apply(sentiment_predict2)
    output_df = pd.concat([df['user'],pred_df['predict_value']],axis=1)
    #output_df = pd.concat([df['contents'],pred_df['predict_value']],axis=1)
    output_df.to_excel('예측2_네이버맵통합.xlsx')
if __name__=='__main__':
    GRU_pos_neg_predict(None,None)

['와', '이런', '것', '도', '상품', '이', '라고', '차라리', '내', '가', '만드', '는', '게', '나을', '뻔']


100%|████████████████████████████████████████████████████████████████████████| 238413/238413 [3:40:57<00:00, 17.98it/s]


In [ ]:
#GRU_pos_neg_predict.py
from utils import *
import re
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()
def GRU_pos_neg_predict(username, prname):
    
    # Create new `pandas` methods which use `tqdm` progress
    # (can use tqdm_gui, optional kwargs, etc.)
    tqdm.pandas()
    #0. 파일 읽기 
    if (username == None) & (prname == None):
        input_directory=''
    else:    
        input_directory = "/".join([username, prname])
    
    #ref: 학습한 모델 이름 (.h5는 제외하고),vocab_size,maxlen
    #ref 주의할점!!!! 각 세개를 띄워서 쓰면 안됨!! 오류남
    #input_: 예측할 별점 없는 데이터 파일 이름 
    #output_ : 예측 점수가 들어가있는 결과 데이터프레임 파일 이름 (/xlsx)
    ref, input_, output_ = Read_Arg_(username, prname, "GRU_pos_neg_predict")
    #1. 메캅설치 확인 (형태소 분석기 어느형태든 상관없음)
    try: 
        mecab = Mecab("C:\mecab\mecab-ko-dic") #자신의 메캅설치 경로 
        print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))
    except :
        print('메캅이 설치되어 있지 않습니다')
    
    #2. 데이터 읽기
    if input_[-3:]=='csv':
        df = pd.read_csv(input_)
    elif input_[-4:]=='xlsx':
        df = pd.read_excel(input_)
    else:
        print('잘못된 인풋파일 형식입니다.')
    df=df.iloc[:98333]
    #3. 학습했던 모델, 토크나이저  불러오기
    ref_list= ref.split(',')
    loaded_model = load_model(ref_list[0])
    with open(ref_list[1], 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    #4. cutoff 가 0.6인 긍부정 예측 함수 
    def sentiment_predict2(new_sentence):
        stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
        new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
        new_sentence = mecab.morphs(new_sentence)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
        pad_new = pad_sequences(encoded, maxlen = int(ref_list[2]))

        score = float(loaded_model.predict(pad_new))
        if(score > 0.6):
            #print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
            return (1)
        else:
            #print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
            #print(score)
            return (0)

    pred_df=pd.DataFrame(columns=['predict_value'])
    df['contents']= df['contents'].astype(str)
    pred_df['predict_value'] = df['contents'].progress_apply(sentiment_predict2)
    output_df = pd.concat([df['user'],pred_df['predict_value']],axis=1)
    #output_df = pd.concat([df['contents'],pred_df['predict_value']],axis=1)
    output_df.to_excel('예측3_네이버맵통합.xlsx')
if __name__=='__main__':
    GRU_pos_neg_predict(None,None)

In [5]:
!jupyter nbconvert GRU_pos_neg_predict.ipynb --to=html --no-input

[NbConvertApp] Converting notebook GRU_pos_neg_predict.ipynb to html
[NbConvertApp] Writing 4982910 bytes to GRU_pos_neg_predict.html
